In [ ]:
import xml.etree.ElementTree as ET
import html  # HTML エンティティのデコード用

def extract_text_from_pages(xml_file, page_numbers=None, tag_name=None):
    """
    指定されたページ番号リストのcontentタグからテキストを抽出する関数。
    
    Args:
        xml_file (str): XMLファイルのパス
        page_numbers (list[int] or None): 抽出対象のページ番号リスト（Noneの場合は全ページ）
        tag_name (str or None): 指定のタグ名（Noneの場合はすべてのタグ）
    
    Returns:
        dict: {ページ番号: [抽出されたテキストのリスト]} の辞書
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()

    extracted_texts = {}

    for page_elem in root.findall("page"):
        page_num = int(page_elem.get("number"))  # ページ番号を取得

        if page_numbers is not None and page_num not in page_numbers:
            continue  # 指定されたページ番号リストにない場合はスキップ

        texts = []
        
        for tag_elem in page_elem.findall("tag"):
            if tag_name is not None and tag_elem.text and tag_name.strip() not in tag_elem.text.strip():
                continue  # 指定のタグ名と一致しない場合はスキップ

            content_elem = tag_elem.find("content")
            if content_elem is not None and content_elem.text:
                # CDATA セクションのデコード
                decoded_text = html.unescape(content_elem.text)

                # CDATA の開始・終了タグを除去
                if decoded_text.startswith("<![CDATA[") and decoded_text.endswith("]]>"):
                    decoded_text = decoded_text[9:-3]

                texts.append(decoded_text)

        if texts:
            extracted_texts[page_num] = texts  # ページ番号をキーにしてテキストを格納

    return extracted_texts

# 例: XMLファイルから page 2, 3, 5 のテキストを抽出
xml_file = 'ir/2025/20020_日清製粉グループ本社/Gx5z.xml'
target_pages = [2, 3, 5]  # 抽出したいページ番号リスト

texts_by_page = extract_text_from_pages(xml_file, page_numbers=target_pages)

# 結果を出力
for page, texts in texts_by_page.items():
    #print(f"Page {page}:")
    for text in texts:
        print(text)
        print("-" * 80)  # 区切り線


In [ ]:
import xml.etree.ElementTree as ET
import html  # HTML エンティティのデコード用

def extract_text_by_tags(xml_file, tag_names=None, page_number=None):
    """
    指定されたタグリストに該当する content タグのテキストを抽出する関数。
    
    Args:
        xml_file (str): XMLファイルのパス
        tag_names (list[str] or None): 抽出対象のタグ名リスト（None の場合はすべてのタグ）
        page_number (int or None): 指定のページ番号（None の場合は全ページ）
    
    Returns:
        dict: {タグ名: [抽出されたテキストのリスト]} の辞書
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()

    extracted_texts = {}

    for page_elem in root.findall("page"):
        if page_number is not None and page_elem.get("number") != str(page_number):
            continue  # 指定されたページ番号と一致しない場合はスキップ

        for tag_elem in page_elem.findall("tag"):
            tag_text = tag_elem.text.strip() if tag_elem.text else ""

            if tag_names is not None and not any(tag in tag_text for tag in tag_names):
                continue  # 指定されたタグリストにない場合はスキップ

            content_elem = tag_elem.find("content")
            if content_elem is not None and content_elem.text:
                # CDATA セクションのデコード
                decoded_text = html.unescape(content_elem.text)

                # CDATA の開始・終了タグを除去
                if decoded_text.startswith("<![CDATA[") and decoded_text.endswith("]]>"):
                    decoded_text = decoded_text[9:-3]

                if tag_text not in extracted_texts:
                    extracted_texts[tag_text] = []
                
                extracted_texts[tag_text].append(decoded_text)

    return extracted_texts

# 例: XMLファイルから特定のタグ情報でテキストを抽出
xml_file = 'ir/2025/20020_日清製粉グループ本社/Gx5z.xml'
target_tags = ["社長メッセージ"]  # 抽出したいタグのリスト

texts_by_tag = extract_text_by_tags(xml_file, tag_names=target_tags)

# 結果を出力
for tag, texts in texts_by_tag.items():
    print(f"Tag: {tag}")
    for text in texts:
        print(text)
        print("-" * 80)  # 区切り線
